<a href="https://colab.research.google.com/github/naokityokoyama/Projetos_Data_Science/blob/main/NLP_Twitter_br.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import numpy as np
import re
import string

In [125]:
df = pd.read_csv('https://raw.githubusercontent.com/minerandodados/mdrepo/master/Tweets_Mg.csv')

In [126]:
df.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,Observação,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['Classificacao'] = df['Classificacao'].replace({'Negativo':0, 'Neutro':np.random.choice([0,1]), 'Positivo':1})

In [8]:
df.describe()

,Unnamed: 0,Geo Coordinates.latitude,Geo Coordinates.longitude,Retweet Count,Classificacao,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
count,8199.000000,104.000000,104.000000,8199.000000,8199.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,4099.000000,-42.964716,-19.530204,43.338212,0.701671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2366.991762,1.399314,1.037815,123.369519,0.457553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,-46.643035,-23.493237,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2049.500000,-43.941907,-19.914265,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4099.000000,-41.971876,-18.879871,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,6148.500000,-41.943662,-18.859150,11.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,8198.000000,-41.560230,-18.568089,1940.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
#lower
frases = df.Text.str.lower()
df.Text = frases

Regex

In [30]:
#emoji
re.findall(r'[�❤️☺⛪]+', df.Text[0])

['���⛪']

In [93]:
#http
re.findall('htt[ps]+://t.co..[a-z0-9]+', df.Text[291])

['https://t.co/julvskxmzz']

In [128]:
'''lista1 = [x for x in df.Text if '#' not in x]
lista2 = [x for x in df.Text if '@' not in x]'''
df.Text = df.Text.apply(lambda x: x.replace('#', ''))

In [129]:
df.Text = df.Text.apply(lambda x: x.replace('@', ''))

In [133]:
lista1 = []
for i in range(len(df.Text)):
  lista1.append(re.sub('htt[ps]+://t.co..[a-z0-9]+','', df.Text[i]))

In [135]:
lista2 = []
for i in range(len(lista1)):
  lista2.append(re.sub(r'[�❤️☺⛪]+', '',lista1[i]))

In [137]:
df.Text = lista2

In [148]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [154]:
tfidf = TfidfVectorizer(lowercase=False)
caracteristicas = tfidf.fit_transform(df.Text)

In [155]:
X = caracteristicas.todense()
y = df.Classificacao

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.32)

In [157]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [158]:
#Score treiro e trest
teste = model.score(X_test, y_test)
treino = model.score(X_train, y_train)
print (f'Acuracia de treino é {treino:.2f} e Acuracia de teste é {teste:.2f} ')

Acuracia de treino é 0.99 e Acuracia de teste é 0.98 


In [159]:
y_pred = model.predict(X_test)


In [164]:
f1 = f1_score(y_test, y_pred)
print (f'O F1 score {f1:.2f}')

O F1 score 0.97


In [161]:
#Dummy
dummy = DummyClassifier()
dummy.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


DummyClassifier(constant=None, random_state=None, strategy='warn')

In [162]:
#Score treiro e trest Dummy
teste = dummy.score(X_test, y_test)
treino = dummy.score(X_train, y_train)
print (f'Acuracia de treino é {treino:.2f} e Acuracia de teste é {teste:.2f} ')

Acuracia de treino é 0.52 e Acuracia de teste é 0.53 
